In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest as iforest
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('consolidate.csv', encoding='big5', index_col=0)

In [3]:
# 排除開盤與收盤時的雜訊
new_df = df[~((df['時間']=='09:00:00') | (df['時間']=='13:15:00'))]

In [4]:
new_df.index=range(len(new_df))

In [5]:
usedColumns = list(new_df.columns)
usedColumns.remove('日期')
usedColumns.remove('時間')

In [6]:
# 訓練集為八月底
endDate=new_df[new_df['日期']=="2019/8/30"].index[-1]+1

In [7]:
rng = np.random.RandomState(123)

In [8]:
# 樣本中異常值的比例
outliers_fraction = 0.015

# 建立幾棵樹
n_estimators = 100

# 抽樣
max_samples = 256

In [9]:
train_X = new_df[usedColumns][0:endDate]
test_X = new_df[usedColumns][endDate:]

In [10]:
train_X.shape

(1424, 45)

In [11]:
# 需要標準化?
# scaler = StandardScaler()
# scaler.fit(train_X)
# transformed_train_X = scaler.transform(train_X)
# transformed_test_X = scaler.transform(test_X)

In [12]:
# isolation forest
clf = iforest(max_samples=max_samples, contamination=outliers_fraction, random_state=rng, n_estimators=n_estimators)
clf.fit(train_X)

IsolationForest(bootstrap=False, contamination=0.015, max_features=1.0,
        max_samples=256, n_estimators=100, n_jobs=1,
        random_state=<mtrand.RandomState object at 0x7fd532aa4510>,
        verbose=0)

In [13]:
y_pred_train = clf.predict(train_X)
y_pred_test = clf.predict(test_X)

train_X_df = new_df[:endDate]
train_X_df['pred'] = y_pred_train

test_X_df = new_df[endDate:]
test_X_df['pred'] = y_pred_test

/usr/lib/python3.4/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/lib/python3.4/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
train_X_df[train_X_df['pred']==-1]['日期'].unique()

array(['2019/4/26', '2019/5/2', '2019/5/6', '2019/5/9', '2019/5/14',
       '2019/5/21', '2019/5/23', '2019/6/19', '2019/6/27', '2019/7/1',
       '2019/7/17', '2019/7/24', '2019/8/1', '2019/8/2', '2019/8/5',
       '2019/8/6', '2019/8/7', '2019/8/15'], dtype=object)

In [15]:
test_X_df[test_X_df['pred']==-1]['日期'].unique()

array(['2019/9/11', '2019/9/16'], dtype=object)

In [16]:
consolidated_df = train_X_df.append(test_X_df)

In [17]:
consolidated_df['outlier']=consolidated_df['pred']==-1

In [19]:
consolidated_df[['日期', '時間', 'outlier']].to_csv('result/if.csv')